**DATA VISUALIZATION HOMEWORK 1**

**NAME:** Letizia 

**SURNAME:** Lanza

**ID:** VR502321

**HOMEWORK:**
- create a chart to show how the relative share of various playlist_genres has changed over time
- create a multiple line chart to show the evolution of the average song properties (only those normalized in the range 0-1 ) over time
- create a small multiple chart with histograms of track popularity for the different genres. Choose an optimal binning
- compare the distributions of the average song duration for the different genres using a visualization idiom for statistical analysis
- create a chart showing the how much the attributes (popularity, danceability, energy, loudness, acousticness, instrumentalness, liveness, valence, tempo) are correlated. Use a proper colormap
- visualize a scatter plot with songs released in 2010, representing loudness and danceability on the x-y coordinates, dots colored by genres and using a different mark (pre-attentive) to make a specific artist of your interest evident